# Ranking Window Functions

These appear to be the most common implementation of Window Functions out in the wild and therefore we shall start here and with the most common of all; ROW_NUMBER.

#### ROW_NUMBER

The reason this is the most commonly used Window Function is because it simply appends an incremental identity value to a dataset.

Therefore if you have a table of data and wish to add an ID column of incremental numbers then this is the way to do it.

Visually, it looks like this:

![RowNumber](https://www.sqlworld.co.uk/images/Blogs/SQLServer/RowNumber.jpg)

As you can see, it literally adds a row number to your data.

So how do we do this and what is the new syntax we require?

Let's start by looking at row_number against our sales.salesOrderHeaderWindow table:

In [ ]:
select row_number() over(order by orderDate) id, *
from JupyterDatabase.sales.salesOrderHeaderWindow;

Now let's explain what we've just seen and done above.

All Window Functions are called with brackets immediately after the function word is used. For some Window Functions we will place something inside these brackets, but we do not do this for row\_number as it will not accept anything being passed in.

We then have OVER. Within the OVER clause we can add ordering. This is how we wish to see the Window Function ordered. In our example above we want the row number to start at the oldest date.

Let's see this again, but this time we will order by saleAmount desc (start our numbering with the highest sales value):

In [ ]:
select row_number() over(order by saleAmount desc) id, *
from JupyterDatabase.sales.salesOrderHeaderWindow;

Note that this ordering is only relevant to the Window Function.  We can still specify an ORDER BY clause in the query to have our data presented how we wish.

We will stick with our row_number being ordered by saleAmount desc, but will return our dataset in date order:

In [ ]:
select row_number() over(order by saleAmount desc) id, *
from JupyterDatabase.sales.salesOrderHeaderWindow
order by orderDate;

As you can see, we can still present the data how we wish whilst maintaining control over the ordering of our Window Function.

However, what if we don't want our row number to be added to the entire table? Maybe we want to add a row number which resets every time we encounter a new salesPersonID?

We can do this as well.  Let's create a row\_number on our table which orders by orderDate (oldest to newest) but resets to 1 each time we encounter a new SalesPersonID:

In [ ]:
select row_number() over(partition by salesPersonID order by orderDate) id, *
from JupyterDatabase.sales.salesOrderHeaderWindow;

The PARTITION BY keywords are used to 'partition' our Window Function so that it acts over each partition of data rather than the table as a whole.

Therefore, in the above example, we are effectively saying:
* Get the data for the first salesPersonID
 * Add a row_number to this data ordered by orderDate
* Get the data for the next salesPersonID
 * Add a row_number to this data ordered by orderDate

And so on until we run out of SalesPersonIDs.

We shall see these being used more and more as we go through other Window Functions.

Note:

We can use the OVER clause in Window Functions in the following configurations:
* ORDER BY only
* PARTITION BY only
* PARTITION BY and ORDER BY

We must have either PARTITION BY or ORDER BY present within the OVER clause, but not necessarily both.

#### RANK

As the name implies, RANK() does exactly what it says on the tin by Ranking your data based on your OVER clause (both ordering and partitioning as required).

Ties are assigned the same ranking value and the next ranking is skipped accordingly.  Therefore if you have three records with the same Rank then the next record would have a Rank of four.

Visually, we can see it as follows:

![Rank](https://www.sqlworld.co.uk/images/Blogs/SQLServer/Rank.jpg)

Let's try this with our sales.salesOrderHeaderWindow table, ordering by orderDate alone (as there are duplicate orderDates present and so we should expect to see the Rank values skip):

In [ ]:
select rank() over(order by orderDate), *
from JupyterDatabase.sales.salesOrderHeaderWindow;

We can see that the Rankings do indeed skip as expected, with 1 appearing twice, therefore skipping to 3, with 9 and 10 simply consecutive.

PARTITION BY can also be used to ensure that we Rank our data by orderDate but resetting our Ranking value each time we change salesPersonID:

In [ ]:
select rank() over(partition by salesPersonID order by orderDate), *
from JupyterDatabase.sales.salesOrderHeaderWindow;

This works as expected and we can see the skipping effect with salesPersonID 51 as they have 2 orders placed on the same day, the 4th January.

#### DENSE_RANK

This function is identical in all ways to RANK except that DENSE_RANK does not skip values when duplicate records are found.

Therefore, where we would expect a duplicate to cause a ranking of 1, 1, 3 with RANK, the DENSE_RANK function will produce 1, 1, 2.

![DenseRank](https://www.sqlworld.co.uk/images/Blogs/SQLServer/DenseRank.jpg)

As this is so similar to RANK we will just go straight into examples, replicating exactly those of RANK above, simply noting the fact that DENSE_RANK does not skip:

In [ ]:
select dense_rank() over(order by orderDate), *
from JupyterDatabase.sales.salesOrderHeaderWindow;

select dense_rank() over(partition by salesPersonID order by orderDate), *
from JupyterDatabase.sales.salesOrderHeaderWindow;

As you can see, the results are exactly as we expected, looking very similar to those of RANK but without any skipping in Ranking values.

#### NTILE

This can be a very helpful function when analysing data as it breaks a dataset up into percentiles (or quartiles, or deciles or 'N' number of tiles, hence the name NTILE).

In this function we can finally insert something into the function's brackets.  We pass in the number of tiles we wish to break our data into.

If we wish to see quartiles we would use NTILE(4), deciles would be NTILE(10) etc.

The calculation that SQL Server uses is:

*Number of rows in each tile = number of rows in result set / number of tiles specified*

Therefore you will see in the following visualisation that we have 8 records and I have requested NTILE(4) which results in 2 records per tile:

![NTile](https://www.sqlworld.co.uk/images/Blogs/SQLServer/NTile.jpg)

The first 2 records are in the first quartile, the last 2 are in the fourth quartile.

Let's try this with our sales.salesOrderHeaderWindow table which has 10 records, therefore we will use NTILE(2) in order to simply place our records in the bottom half or top half of our data based on saleAmount:

In [ ]:
select ntile(2) over(order by saleAmount), *
from JupyterDatabase.sales.salesOrderHeaderWindow;

We can also PARTITION this function as we do with all others.  However, we only declare NTILE once and therefore any tile number we provide will be used for all partitions.

As we have 3 sales for one salesPersonID and 7 for the other, we will use NTILE(3).  This will provide a single record per tile for salesPersonID 44 and will place 2 or 3 records per tile for salesPersonID 51:

In [ ]:
select ntile(3) over(partition by salesPersonID order by saleAmount), *
from JupyterDatabase.sales.salesOrderHeaderWindow;

This works exactly as we expect it to, with tile 1 being allocated the 3 records for salesPersonID 51 while the other tiles both receive 2.